# Building your first Agent Systems with Databricks

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/llm-tools-functions/ai-agent-functions.png?raw=true" style="float: right" width="700px">
LLMs are powerful to answer general knowledge, but don't have any information on your own business.

Databricks makes it easy to develop custom tools which can be called by your LLM, creating a complete agent with reasoning capability.


### Build Simple UC Tools
In this notebook, we'll register the following to Unity Catalogs:
- **SQL Functions**: Create queries that access customer information, order and subscriptions.
- **Simple Python Function**: Create and register a Python function doing math operations to overcome some common limitations of language models.

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=796524194907820&notebook=%2F01-create-tools%2F01_create_first_billing_agent&demo_name=ai-agent&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fai-agent%2F01-create-tools%2F01_create_first_billing_agent&version=1">

In [0]:
%pip install databricks-agents mlflow>=3.1.0 databricks-sdk==0.55.0 unitycatalog-ai[databricks]
# Restart to load the packages into the Python environment
dbutils.library.restartPython()

In [0]:
%run ../_resources/01-setup

Generate synthetic data we're going to use with our agents.


## 1. Get the customer details based on their email

Let's add a function to retrieve a customer detail based on their email.

*Note: realtime postgres tables could be used to provide low latencies for point-queries like these. However, to keep this demo simple, we'll use a SQL endpoint. Using a SQL endpoint is also a very good way to provide analytics capabilities to your agent!*


In [0]:
%sql
CREATE OR REPLACE FUNCTION get_customer_by_email(email_input STRING COMMENT 'customer email used to retrieve customer information')
RETURNS TABLE (
    customer_id BIGINT,
    first_name STRING,
    last_name STRING,
    email STRING,
    phone STRING,
    address STRING,
    city STRING,
    state STRING,
    zip_code STRING,
    customer_segment STRING,
    registration_date DATE,
    customer_status STRING,
    loyalty_tier STRING,
    tenure_years DOUBLE,
    churn_risk_score BIGINT,
    customer_value_score BIGINT
)
COMMENT 'Returns the customer record matching the provided email address. Includes its ID, firstname, lastname and more.'
RETURN (
    SELECT * FROM customers
    WHERE email = email_input
    LIMIT 1
);


In [0]:
%sql SELECT * FROM get_customer_by_email('john21@example.net');


## 2. Retrieve all billing informations
Let's add a function to get all the customer orders and subscriptions. This function will take a customer ID as input and return som aggregation to filter all the past billing informations and current subscriptions.

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_customer_billing_and_subscriptions(customer_id_input BIGINT COMMENT 'customer ID used to retrive orders, billing and subscriptiosn')
RETURNS TABLE (
    customer_id BIGINT,
    subscription_id BIGINT,
    service_type STRING,
    plan_name STRING,
    plan_tier STRING,
    monthly_charge BIGINT,
    start_date DATE,
    contract_length_months BIGINT,
    status STRING,
    autopay_enabled BOOLEAN,
    total_billed DOUBLE,
    total_paid DOUBLE,
    total_late_payments BIGINT,
    total_late_fees DOUBLE,
    latest_payment_status STRING
)
COMMENT 'Returns subscription and billing details for a customer.'
RETURN (
    SELECT
        s.customer_id, s.subscription_id, s.service_type, s.plan_name, s.plan_tier,
        s.monthly_charge, s.start_date, s.contract_length_months, s.status, s.autopay_enabled,
        COALESCE(b.total_billed, 0), COALESCE(b.total_paid, 0),
        COALESCE(b.total_late_payments, 0), COALESCE(b.total_late_fees, 0),
        COALESCE(b.latest_payment_status, 'N/A')
    FROM subscriptions s
    LEFT JOIN (
        SELECT
            subscription_id, customer_id,
            SUM(total_amount) AS total_billed,
            SUM(payment_amount) AS total_paid,
            COUNT_IF(payment_date > due_date OR payment_status = 'Late') AS total_late_payments,
            SUM(CASE WHEN payment_date > due_date OR payment_status = 'Late' THEN total_amount - payment_amount ELSE 0 END) AS total_late_fees,
            MAX(payment_status) AS latest_payment_status
        FROM billing
        WHERE customer_id = customer_id_input
        GROUP BY subscription_id, customer_id
    ) b ON s.subscription_id = b.subscription_id
    WHERE s.customer_id = customer_id_input
);

In [0]:
%sql
SELECT *
FROM get_customer_billing_and_subscriptions(
  (SELECT customer_id FROM get_customer_by_email('john21@example.net'))
);



## 3. Give the LLM a Python Function to compute Math
LLMs typically struggle to run any advance math. Let's add a tool to let the LLM compute any math expression, using python directly.

Databricks makes it easy, running safe, sandboxed python functions:

In [0]:
# -----------------------
# TOOL 2: evaluate math expression
# -----------------------
def calculate_math_expression(expression: str) -> float:
    """
    Evaluates a basic math expression safely.

    Args:
        expression (str): A math expression (e.g., "sqrt(2 + 3 * (4 - 1)), using python math functions.").

    Returns:
        float: The result of the evaluated expression.
    """
    import math
    allowed_names = {k: v for k, v in math.__dict__.items() if not k.startswith("__")}
    allowed_names.update({"abs": abs, "round": round})

    try:
        result = eval(expression, {"__builtins__": None}, allowed_names)
        return float(result)
    except Exception as e:
        raise ValueError(f"Invalid expression: {expression}. Error: {str(e)}")

print(calculate_math_expression("6 + 3 * (13 - 1)"))

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

# this will deploy the tool to UC, automatically setting the metadata in UC based on the tool's docstring & typing hints
python_tool_uc_info = client.create_python_function(func=calculate_math_expression, catalog=catalog, schema=dbName, replace=True)

# the tool will deploy to a function in UC called `{catalog}.{schema}.{func}` where {func} is the name of the function
# Print the deployed Unity Catalog function name
print(f"Deployed Unity Catalog function name: {python_tool_uc_info.full_name}")
# Create HTML link to created functions
displayHTML(f'<a href="/explore/data/functions/{catalog}/{dbName}/calculate_math_expression" target="_blank">Go to Unity Catalog to see Registered Functions</a>')

###### Note: There is also a function registered in System.ai.python_exec that will let your LLM run generated code in a sandboxed environment:

```

%sql
SELECT system.ai.python_exec("""
from datetime import datetime
print(datetime.now().strftime('%Y-%m-%d'))
""") as current_date

```



## 4: Let's go over to the AI Playground to see how we can use these functions and assemble our first Agent!

Open the [Playground](/ml/playground) and select the tools we created to test your agent!

<div style="float: right; width: 70%;">
  <img 
    src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/\
cross_demo_assets/AI_Agent_GIFs/AI_agent_function_selection.gif" 
    alt="Function Selection" 
    width="100%"
  >
</div>

### Location Guide

Your functions are organized in Unity Catalog using this structure:

#### Example Path:
`my_catalog.my_schema.my_awesome_function`

💡 Note: Replace the example names with your actual catalog and schema names.

## What's next: Evaluation

Our agent is now ready and leveraging our tools to properly answer our questions.

But how can we make sure it's working properly, and more importantly will still work well for future questions and modifications?

To do so, we need to build an Evaluation dataset and leverage MLFlow to automatically analyze our agent!

Open the [02_agent_eval/02.1_agent_evaluation]($../02_agent_eval/02.1_agent_evaluation) notebook to see how to deploy your agent using Langchain and run your first evaluations!